In [1]:
from pyspark.sql import SparkSession
import os

spark_version = '3.4.1'
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:{spark_version},org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} pyspark-shell'

import findspark
findspark.init(os.environ['SPARK_HOME'] )

spark = (SparkSession
        .builder
        .appName('app')
        .getOrCreate())

In [2]:
spark

In [3]:
import pandas as pd

spark.createDataFrame(pd.Series(list(range(100))).to_frame()).show()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+---+
|  0|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



In [24]:
!python /home/jovyan/work/producer.py

^C
Traceback (most recent call last):
  File "/home/jovyan/work/producer.py", line 29, in <module>
    produce()
  File "/home/jovyan/work/producer.py", line 26, in produce
    sleep(wait)
KeyboardInterrupt


In [8]:
kafka_servers='localhost:9092'
kafka_servers='broker_kafka:29092'
topic= 'FirstTopic'

In [25]:


stream = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_servers) \
    .option("kafka.security.protocol", "PLAINTEXT") \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [26]:
checkpoint_path = "/home/jovyan/work/checkpoint/"

stream.writeStream\
    .format("parquet") \
    .option("checkpointLocation", checkpoint_path) \
    .option("path", "/home/jovyan/work/dir/")\
    .start()

In [16]:
stream\
    .writeStream \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 second") \
    .start()

In [6]:
stream = spark.read.format("kafka") \
    .option("kafka.bootstrap.servers", 'broker_kafka:29092') \
    .option("kafka.security.protocol", "PLAINTEXT") \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
stream.show()

+----+--------------------+----------+---------+------+--------------------+-------------+
| key|               value|     topic|partition|offset|           timestamp|timestampType|
+----+--------------------+----------+---------+------+--------------------+-------------+
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     0|2023-10-29 16:52:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     1|2023-10-29 16:52:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     2|2023-10-29 16:52:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     3|2023-10-29 16:52:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     4|2023-10-29 16:52:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     5|2023-10-29 16:54:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     6|2023-10-29 16:54:...|            0|
|null|[7B 22 66 69 72 7...|FirstTopic|        0|     7|2023-10-29 16:54:...|            0|

In [20]:
from kafka import KafkaProducer
from faker import Faker
import numpy as np
import json
from time import sleep


topic_name = 'FirstTopic'
producer = KafkaProducer(bootstrap_servers='broker_kafka:29092')
_instance = Faker()


_data = {
    "first_name": _instance.first_name(),
    "city":_instance.city(),
    "phone_number":_instance.phone_number(),
    "state":_instance.state(),
    "id":str('_')
}
_payload = json.dumps(_data).encode("utf-8")
response = producer.send(topic_name, _payload)
#print(response)

#wait = np.random.randint(3,15,1)[0]
#sleep(wait)




In [21]:
print(response)

In [ ]:
try:
    from kafka import KafkaConsumer
    import json
    import requests
    import os
    import sys
except Exception as e:
    pass

# ------------------------------
os.environ['KAFKA_TOPIC'] = "FirstTopic"
os.environ['SERVER_END_POINT'] = "broker_kafka:29092"
# -----------------------------



class ElasticSearchKafkaUploadRecord:

    def __init__(self, json_data, hash_key, index_name):
        self.hash_key = hash_key
        self.json_data = json_data
        self.index_name = index_name.lower()

    def upload(self):
        """
        Uploads records on Elastic Search Cluster
        :return
        """
        URL = "{}/{}/_doc/{}".format(
            os.getenv("SERVER_END_POINT"), self.index_name, self.hash_key
        )
        print(URL)

        headers = {"Content-Type": "application/json"}

        response = requests.request(
            "PUT", URL, headers=headers, data=json.dumps(self.json_data),
        )
        print(response)

        return {"status": 200, "data": {"message": "record uploaded to Elastic Search"}}


def main():

    consumer = KafkaConsumer(os.getenv("KAFKA_TOPIC"))

    for msg in consumer:

        payload = json.loads(msg.value)
        payload["meta_data"]={
            "topic":msg.topic,
            "partition":msg.partition,
            "offset":msg.offset,
            "timestamp":msg.timestamp,
            "timestamp_type":msg.timestamp_type,
            "key":msg.key,
        }

        print(payload)

        # helper = ElasticSearchKafkaUploadRecord(json_data=payload,
        #                                         index_name=payload.get("meta_data").get("topic"),
        #                                         hash_key=payload.get("meta_data").get("offset"),
        #                                         )
        # response = helper.upload()


main()